<a href="https://colab.research.google.com/github/aileenvl/aiagentweekmty/blob/main/AIAgentWeekMTY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L2: Create Agents to Research and Write an Article
In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
!pip install requests

In [ ]:
pip install 'crewai[tools]'

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install langchain-google-genai crewai

Import from the crewAI libray.

In [ ]:
import os
from crewai import Agent, Task, Crew, LLM
from langchain.tools import Tool
import requests
import json

# Configurar la API key de Gemini
os.environ["GEMINI_API_KEY"] = "your-api-key"
os.environ["DEVTO_API_KEY"] = "your-api-key"

# Crear el LLM usando la forma correcta según la documentación
llm = LLM(
    model="gemini/gemini-2.0-flash-exp",  # Formato correcto con el prefijo "gemini/"
    temperature=0.7,
    api_key=os.environ["GEMINI_API_KEY"]
)


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

In [ ]:
from crewai.tools import BaseTool, tool
from pydantic import BaseModel, Field
from typing import Type, List, Optional

# Input model for the Dev.to publishing tool
class DevtoPublishInput(BaseModel):
    """Input schema for publishing to Dev.to."""
    title: str = Field(..., description="Article title")
    content: str = Field(..., description="Content in Markdown format")
    tags: Optional[List[str]] = Field(None, description="List of tags (maximum 4)")
    series: Optional[str] = Field(None, description="Series name if it's part of one")
    published: bool = Field(False, description="True to publish, False for draft")

# Custom tool for publishing to Dev.to
class DevtoPublishTool(BaseTool):
    name: str = "publish_to_devto"
    description: str = "Publishes an article to Dev.to using its API"
    args_schema: Type[BaseModel] = DevtoPublishInput

    def _run(
        self,
        title: str,
        content: str,
        tags: Optional[List[str]] = None,
        series: Optional[str] = None,
        published: bool = False
    ) -> str:
        """Publishes an article to Dev.to."""
        api_key = os.environ.get("DEVTO_API_KEY")
        if not api_key:
            return "Error: Dev.to API key not configured. Set up DEVTO_API_KEY."

        # Prepare article data
        article_data = {
            "article": {
                "title": title,
                "body_markdown": content,
                "published": published
            }
        }

        # Add tags if provided (maximum 4 allowed)
        if tags:
            if len(tags) > 4:
                tags = tags[:4]  # Dev.to allows maximum 4 tags
            article_data["article"]["tags"] = tags

        # Add series if provided
        if series:
            article_data["article"]["series"] = series

        # Send request to Dev.to API
        headers = {
            "api-key": api_key,
            "Content-Type": "application/json"
        }

        try:
            response = requests.post(
                "https://dev.to/api/articles",
                json=article_data,
                headers=headers
            )

            if response.status_code not in [200, 201]:
                return f"Error publishing: {response.status_code} - {response.text}"

            result = response.json()
            article_url = result.get('url', 'Not available')
            return f"Article {'published' if published else 'saved as draft'} successfully. URL: {article_url}"
        except Exception as e:
            return f"Error connecting to Dev.to API: {str(e)}"

# Alternative using the @tool decorator
@tool("Publish to Dev.to")
def publish_to_devto_tool(
    title: str,
    content: str,
    tags: Optional[List[str]] = None,
    series: Optional[str] = None,
    published: bool = False
) -> str:
    """
    Publishes an article to Dev.to using its API.

    Args:
        title: Article title
        content: Content in Markdown format
        tags: List of tags (maximum 4)
        series: Series name if it's part of one
        published: True to publish, False for draft
    """
    api_key = os.environ.get("DEVTO_API_KEY")
    if not api_key:
        return "Error: Dev.to API key not configured. Set up DEVTO_API_KEY."

    # Prepare article data
    article_data = {
        "article": {
            "title": title,
            "body_markdown": content,
            "published": published
        }
    }

    # Add tags if provided (maximum 4 allowed)
    if tags and len(tags) > 0:
        if len(tags) > 4:
            tags = tags[:4]  # Dev.to allows maximum 4 tags
        article_data["article"]["tags"] = tags

    # Add series if provided
    if series:
        article_data["article"]["series"] = series

    # Send request to Dev.to API
    headers = {
        "api-key": api_key,
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(
            "https://dev.to/api/articles",
            json=article_data,
            headers=headers
        )

        if response.status_code not in [200, 201]:
            return f"Error publishing: {response.status_code} - {response.text}"

        result = response.json()
        article_url = result.get('url', 'Not available')
        return f"Article {'published' if published else 'saved as draft'} successfully. URL: {article_url}"
    except Exception as e:
        return f"Error connecting to Dev.to API: {str(e)}"

devto_tool = DevtoPublishTool()

In [ ]:
publisher = Agent(
    role="Dev.to Publisher",
    goal="Format and publish technical articles on Dev.to",
    backstory="You're a technical content strategist specializing in developer-focused content. You know how to format and optimize articles for the Dev.to platform to maximize engagement and visibility in the developer community.",
    llm=llm,
    verbose=True,
    tools=[devto_tool]
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan


In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
publish = Task(
    description="""
    Prepare and publish the edited article on Dev.to. You must:
    1. Optimize the title for the developer audience
    2. Format the content properly in Markdown
    3. Select up to 4 relevant tags
    4. Decide if the article should be part of a series
    5. Save as a draft (published=False) initially

    Use the publish_to_devto tool to submit the article to Dev.to.
    """,
    expected_output="""
    A confirmation message that the article has been successfully published to Dev.to as a draft,
    including the title, tags selected, and URL of the article if available.

    The message should include:
    1. The optimized title used
    2. The tags selected (max 4)
    3. Whether it was added to a series
    4. Confirmation of successful publication as a draft
    """,
    agent=publisher,
    context=[edit]
)


Creating the Crew
Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor, publisher],
    tasks=[plan, write, edit, publish],
    verbose=True
)

Running the Crew
Note: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "AI agents"})

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import display, Markdown

# Si result es un objeto CrewOutput, accede a su atributo 'raw'
if hasattr(result, 'raw'):
    display(Markdown(result.raw))
else:
    # Intenta extraer el contenido del markdown de la representación en string
    content = str(result)
    # Si el contenido comienza con marcadores de código markdown, elimínalos
    if content.startswith("```markdown"):
        content = content[len("```markdown"):].strip()
    if content.endswith("```"):
        content = content[:-3].strip()

    display(Markdown(content))

In [ ]:
print(result)

Try it Yourself
Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = ""
result = crew.kickoff(inputs={"topic": topic})